# 🎙️ Voice Conversational Agentic AI - Colab API Simulation Version (Updated for LangChain v0.1+)

In [1]:
!pip install -U langchain langchain-community langchain-openai openai faiss-cpu speechrecognition gtts pydub --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [2]:
import openai
openai.api_key = "# Replace with your key"

In [3]:
import pandas as pd
from google.colab import files
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import time

# Upload dataset
uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))

# Chunk into documents
text_data = '\n'.join(df.astype(str).apply(lambda row: ' '.join(row), axis=1).tolist())
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents([text_data])

# Create embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

# Split into smaller batches
chunk_size = 3
batches = [docs[i:i + chunk_size] for i in range(0, len(docs), chunk_size)]

vectorstore = None

for i, batch in enumerate(batches):
    print(f"Embedding batch {i+1}/{len(batches)}...")
    try:
        partial_store = FAISS.from_documents(batch, embeddings)
        if vectorstore is None:
            vectorstore = partial_store
        else:
            vectorstore.merge_from(partial_store)
    except Exception as e:
        print(f"Error in batch {i+1}: {e}")
    print("Sleeping for 3 seconds to respect rate limits...")
    time.sleep(3)  # Increase to  3 seconds


Saving HackathonInternalKnowledgeBase.csv to HackathonInternalKnowledgeBase.csv
Embedding batch 1/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 2/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 3/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 4/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 5/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 6/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 7/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 8/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 9/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 10/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 11/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 12/25...
Sleeping for 3 seconds to respect rate limits...
Embedding batch 13/25...
Sleeping for 3 s

HackathonInternalKnowledgeBase.csv(text/csv) - 33365 bytes, last modified: 7/13/2025 - 100% done
Saving HackathonInternalKnowledgeBase.csv to HackathonInternalKnowledgeBase.csv
---------------------------------------------------------------------------
RateLimitError                            Traceback (most recent call last)
/tmp/ipython-input-3-3028965663.py in <cell line: 0>()
     16 # Create embeddings
     17 embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)
---> 18 vectorstore = FAISS.from_documents(docs, embeddings)

6 frames
/usr/local/lib/python3.11/dist-packages/openai/_base_client.py in request(self, cast_to, options, stream, stream_cls)
   1042
   1043                 log.debug("Re-raising status error")
-> 1044                 raise self._make_status_error_from_response(err.response) from None
   1045
   1046             break

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [4]:
conversation_memory = []

def get_rag_context(query):
    docs = vectorstore.similarity_search(query, k=3)
    return '\n'.join([doc.page_content for doc in docs])

def chat_with_llm(user_input):
    rag_context = get_rag_context(user_input)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        *conversation_memory,
        {"role": "user", "content": f"{user_input}\n\nRelevant info:\n{rag_context}"}
    ]
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    reply = response.choices[0].message['content']
    conversation_memory.append({"role": "user", "content": user_input})
    conversation_memory.append({"role": "assistant", "content": reply})
    return reply

In [5]:
import speech_recognition as sr
import time

def transcribe_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio = recognizer.listen(source)
        print("Processing...")
        try:
            start = time.time()
            text = recognizer.recognize_google(audio)
            duration = time.time() - start
            print("You said:", text)
            return {"transcription": text, "duration_seconds": round(duration, 2)}
        except Exception as e:
            print("Error:", str(e))
            return {"transcription": "", "duration_seconds": 0}

In [6]:
from gtts import gTTS
from IPython.display import Audio

def speak_text(text):
    start = time.time()
    tts = gTTS(text)
    tts.save("response.mp3")
    duration = time.time() - start
    return {"audio": Audio("response.mp3"), "tts_duration": round(duration, 2)}

In [7]:
def transcribe_endpoint():
    return transcribe_audio()

def chat_endpoint(message):
    response = chat_with_llm(message)
    return {"response": response, "conversation": conversation_memory}

def speak_endpoint(text):
    return speak_text(text)

def converse_endpoint():
    transcribed = transcribe_audio()
    if not transcribed["transcription"]:
        return {"error": "Could not transcribe"}
    response = chat_with_llm(transcribed["transcription"])
    audio_data = speak_text(response)
    return {
        "transcribed_text": transcribed["transcription"],
        "response": response,
        "tts_duration": audio_data["tts_duration"],
        "audio": audio_data["audio"]
    }

def reset_endpoint():
    global conversation_memory
    conversation_memory = []
    return {"status": "conversation memory cleared"}

def upload_rag_docs():
    uploaded = files.upload()
    filename = next(iter(uploaded))
    df = pd.read_csv(filename)
    text_data = '\n'.join(df.astype(str).apply(lambda row: ' '.join(row), axis=1).tolist())
    docs = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).create_documents([text_data])
    global vectorstore
    vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings(openai_api_key=openai.api_key))
    return {"status": f"{filename} uploaded and indexed", "num_chunks": len(docs)}

In [9]:
!apt-get install ffmpeg -y
!pip install pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
from gtts import gTTS
from IPython.display import Audio
import time

def speak_text(text):
    start = time.time()
    tts = gTTS(text)
    tts.save("response.mp3")
    duration = time.time() - start
    return {"audio": Audio("response.mp3"), "tts_duration": round(duration, 2)}

In [11]:
import IPython
from IPython.display import Javascript
from google.colab import output
import base64

def record_audio(filename='recorded.wav', duration=5):
    js_code = """
    const sleep = time => new Promise(resolve => setTimeout(resolve, time));
    const b2text = blob => new Promise(resolve => {
      const reader = new FileReader();
      reader.onloadend = () => resolve(reader.result);
      reader.readAsDataURL(blob);
    });
    async function record() {
      const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
      const recorder = new MediaRecorder(stream);
      const data = [];
      recorder.ondataavailable = event => data.push(event.data);
      recorder.start();
      await sleep(""" + str(5 * 1000) + """);
      recorder.stop();
      await new Promise(resolve => recorder.onstop = resolve);
      const blob = new Blob(data, { type: 'audio/wav' });
      const base64 = await b2text(blob);
      google.colab.kernel.invokeFunction('notebook.save_audio', [base64], {});
    }
    record();
    """
    display(Javascript(js_code))

def save_audio(base64_wav):
    wav_data = base64.b64decode(base64_wav.split(',')[1])
    with open("recorded.wav", "wb") as f:
        f.write(wav_data)
    print("Audio saved as 'recorded.wav'")

output.register_callback('notebook.save_audio', save_audio)

In [12]:
from pydub import AudioSegment

def convert_to_wav(input_path="recorded.wav", output_path="converted.wav"):
    audio = AudioSegment.from_file(input_path)
    audio.export(output_path, format="wav")
    print(f" Converted to proper WAV: {output_path}")
    return output_path

In [13]:
import speech_recognition as sr

def transcribe_audio_file(filename='converted.wav'):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        print("Transcription:", text)
        return {"transcription": text, "duration_seconds": None}
    except Exception as e:
      import traceback
      print("Full Error Traceback:")
      traceback.print_exc()

In [15]:
# 1. Record audio (will prompt in browser)
record_audio(duration=5)

<IPython.core.display.Javascript object>

✅ Audio saved as 'recorded.wav'


In [16]:
wav_path = convert_to_wav("recorded.wav")

✅ Converted to proper WAV: converted.wav


In [17]:
transcribed = transcribe_audio_file(wav_path)

📝 Transcription: what is the associate one name for 36th Street


In [18]:
import os
print("File exists:", os.path.exists("converted.wav"))
print("File size (bytes):", os.path.getsize("converted.wav"))

✅ File exists: True
📦 File size (bytes): 887084


In [19]:
from IPython.display import Audio
Audio("converted.wav")